### Mounting the container into Databricks

In [0]:
# Install required packages
%pip install azure-identity azure-keyvault-secrets

  Obtaining dependency information for azure-identity from https://files.pythonhosted.org/packages/f0/d5/3995ed12f941f4a41a273d9b1709282e825ef87ed8eab3833038fee54d59/azure_identity-1.19.0-py3-none-any.whl.metadata
  Using cached azure_identity-1.19.0-py3-none-any.whl.metadata (80 kB)
  Obtaining dependency information for azure-keyvault-secrets from https://files.pythonhosted.org/packages/bf/ad/e5dd4c09ed80196b1b35f107502b12e32d06eb2d965adf4673df0d5cf85e/azure_keyvault_secrets-4.9.0-py3-none-any.whl.metadata
  Using cached azure_keyvault_secrets-4.9.0-py3-none-any.whl.metadata (29 kB)
  Obtaining dependency information for azure-core>=1.31.0 from https://files.pythonhosted.org/packages/39/83/325bf5e02504dbd8b4faa98197a44cdf8a325ef259b48326a2b6f17f8383/azure_core-1.32.0-py3-none-any.whl.metadata
  Using cached azure_core-1.32.0-py3-none-any.whl.metadata (39 kB)
  Obtaining dependency information for msal>=1.30.0 from https://files.pythonhosted.org/packages/30/7c/489cd931a752d05753d730e8

In [0]:
# Restart Python interpreter to ensure new packages are loaded
%restart_python

In [0]:
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

# Key Vault configuration
key_vault_url = "https://Olist-Key.vault.azure.net/"
credential = DefaultAzureCredential()
client = SecretClient(vault_url=key_vault_url, credential=credential)

# Retrieve secrets from Key Vault
client_id = client.get_secret("olist-client-id").value
client_secret = client.get_secret("olist-client-secret").value
tenant_id = client.get_secret("olist-tenant-id").value

# Unmount the existing mount point if it exists
dbutils.fs.unmount("/mnt/olist-store-data")

# Create the configurations
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": client_id,
    "fs.azure.account.oauth2.client.secret": client_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"
}

# Mount the storage
dbutils.fs.mount(
    source="abfss://olist-store-data@olistbrdata.dfs.core.windows.net",
    mount_point="/mnt/olist-store-data",
    extra_configs=configs
)

/mnt/olist-store-data has been unmounted.


True

### Check mounting of the storage-account container

In [0]:
# Check if the mounting is successful or not
dbutils.fs.ls("/mnt/olist-store-data")

[FileInfo(path='dbfs:/mnt/olist-store-data/raw-data/', name='raw-data/', size=0, modificationTime=1735461319000),
 FileInfo(path='dbfs:/mnt/olist-store-data/ready-data/', name='ready-data/', size=0, modificationTime=1735792345000),
 FileInfo(path='dbfs:/mnt/olist-store-data/test-upload/', name='test-upload/', size=0, modificationTime=1736860622000),
 FileInfo(path='dbfs:/mnt/olist-store-data/transformed-data/', name='transformed-data/', size=0, modificationTime=1735461344000)]

### Read product_category dataset from raw-data folder

In [0]:
product_category = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olist-store-data/raw-data/product_category_name_translation.csv")

In [0]:
product_category.show(10)
product_category.printSchema()
display(product_category.limit(10))

+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|         beleza_saude|                health_beauty|
| informatica_acess...|         computers_accesso...|
|           automotivo|                         auto|
|      cama_mesa_banho|               bed_bath_table|
|     moveis_decoracao|              furniture_decor|
|        esporte_lazer|               sports_leisure|
|           perfumaria|                    perfumery|
| utilidades_domest...|                   housewares|
|            telefonia|                    telephony|
|   relogios_presentes|                watches_gifts|
+---------------------+-----------------------------+
only showing top 10 rows

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



product_category_name,product_category_name_english
beleza_saude,health_beauty
informatica_acessorios,computers_accessories
automotivo,auto
cama_mesa_banho,bed_bath_table
moveis_decoracao,furniture_decor
esporte_lazer,sports_leisure
perfumaria,perfumery
utilidades_domesticas,housewares
telefonia,telephony
relogios_presentes,watches_gifts


### Data cleaning for product_catergory dataset
- **Step 1**: Basic text cleaning (lowercase, trim, special character removal).

- **Step 2**: Title case conversion for English category names.

- **Step 3**: Plural form detection and consistency checking.

- **Step 4**: Special case handling for specific categories.

- **Step 5**: Similarity calculation between Portuguese and English names.

In [0]:
# This script performs data cleaning and analysis on the product_category dataset.
# It standardizes category names, checks for consistency between Portuguese and English versions,
# and identifies potential mismatches or inconsistencies.

# Import libraries
from pyspark.sql.functions import (
    col, lower, when, levenshtein, count as count_, 
    sum as sum_, greatest, length, regexp_replace, trim,
    initcap  # Add initcap for title case conversion
)

# Define helper function to check if a category name is likely to be plural
def is_likely_plural(column_name):
    return when(
        (lower(col(column_name)).endswith('s') & 
         ~lower(col(column_name)).endswith('ss') &
         ~lower(col(column_name)).isin('business', 'press', 'express')) |
        (lower(col(column_name)).endswith('is') |
         lower(col(column_name)).endswith('es') |
         lower(col(column_name)).endswith('ns')),
        True
    ).when(
        lower(col(column_name)).isin(
            'alimentos', 'bebes', 'artes',
            'housewares', 'supplies', 'clothes'
        ),
        True
    ).otherwise(False)

# Define function to apply specific mappings and rules for special cases in category names
def apply_category_mapping(df):
    return df.withColumn(
        "is_plural_consistent",
        when(
            # Handle Portuguese-English special cases
            (lower(col("product_category_name_clean")).isin(
                "alimentos_bebidas", "bebes", "artes", "alimentos"
            )) & 
            (lower(col("product_category_name_english_clean")).isin(
                "food_drink", "baby", "art", "food"
            )),
            True
        ).when(
            # Handle tools/ferramentas pattern
            (col("product_category_name_clean").like("%ferramentas%")) & 
            (col("product_category_name_english_clean").like("%tools%")),
            True
        ).when(
            # Handle supplies pattern
            col("product_category_name_english_clean").like("%supplies%"),
            True
        ).when(
            # Handle clothes pattern
            col("product_category_name_english_clean").like("%clothes%"),
            True
        ).when(
            # Handle books pattern
            (col("product_category_name_clean").like("%livros_%")) & 
            (col("product_category_name_english_clean").like("books_%")),
            True
        ).when(
            # Handle business/commerce terms
            (col("product_category_name_clean").like("%comercio%")) & 
            (col("product_category_name_english_clean").like("%commerce%")),
            True
        ).when(
            # Handle industry/business terms
            (col("product_category_name_clean").like("%negocios%")) & 
            (col("product_category_name_english_clean").like("%business%")),
            True
        ).otherwise(col("is_plural_consistent"))
    )

# Define function to calculate similarity between Portuguese and English category names
def calculate_similarity(df):
    # First normalize common terms
    normalized_df = df.withColumn(
        "normalized_pt",
        when(col("product_category_name_clean").like("%portateis_casa_forno_e_cafe%"), 
             "small_appliances_home_oven_and_coffee")
        .when(col("product_category_name_clean").like("%fashion_roupa_infanto_juvenil%"),
              "fashion_childrens_clothes")
        .when(col("product_category_name_clean").like("%utilidades_domesticas%"),
              "housewares")
        .when(col("product_category_name_clean").like("%instrumentos_musicais%"),
              "musical_instruments")
        .when(col("product_category_name_clean").like("%livros_interesse_geral%"),
              "books_general_interest")
        .when(col("product_category_name_clean").like("%moveis_cozinha_area_de_servico_jantar_e_jardim%"), 
             "kitchen_dining_laundry_garden_furniture")
        .when(col("product_category_name_clean").like("%moveis_colchao_e_estofado%"),
              "furniture_mattress_and_upholstery")
        .when(col("product_category_name_clean").like("%moveis%"), 
             regexp_replace(col("product_category_name_clean"), "moveis", "furniture"))
        .when(col("product_category_name_clean").like("%construcao_ferramentas_ferramentas%"),
              "construction_tools_tools")
        .when(col("product_category_name_clean").like("%construcao_ferramentas%"),
              regexp_replace(
                  regexp_replace(col("product_category_name_clean"), 
                               "construcao", "construction"),
                  "ferramentas", "tools"
              ))
        .otherwise(col("product_category_name_clean"))
    ).withColumn(
        "normalized_en",
        when(col("product_category_name_english_clean").like("%costruction%"),
             regexp_replace(col("product_category_name_english_clean"), "costruction", "construction"))
        .otherwise(col("product_category_name_english_clean"))
    )
    
    return normalized_df.withColumn(
        "name_similarity",
        when(
            (col("normalized_pt") == col("normalized_en")),
            0
        ).when(
            (col("product_category_name_clean").isin(
                "portateis_casa_forno_e_cafe",
                "fashion_roupa_infanto_juvenil",
                "utilidades_domesticas",
                "instrumentos_musicais",
                "livros_interesse_geral"
            )),
            5
        ).otherwise(
            levenshtein(
                col("normalized_pt"),
                col("normalized_en")
            )
        )
    )

# Main cleaning function
def clean_product_categories(df):
    # Basic cleaning
    cleaned_df = df.withColumn(
        "product_category_name_clean",
        regexp_replace(lower(trim(col("product_category_name"))), "[^a-z0-9_]", "_")
    ).withColumn(
        "product_category_name_english_clean",
        regexp_replace(lower(trim(col("product_category_name_english"))), "[^a-z0-9_]", "_")
    )
    
    # Convert English category names to title case
    cleaned_df = cleaned_df.withColumn(
        "product_category_name_english_title",
        initcap(regexp_replace(col("product_category_name_english_clean"), "_", " "))
    )
    
    # Check for plural consistency
    cleaned_df = cleaned_df.withColumn(
        "is_original_plural",
        is_likely_plural("product_category_name_clean")
    ).withColumn(
        "is_english_plural",
        is_likely_plural("product_category_name_english_clean")
    )
    
    # Create is_plural_consistent column
    cleaned_df = cleaned_df.withColumn(
        "is_plural_consistent",
        col("is_original_plural") == col("is_english_plural")
    )
    
    # Apply category mapping and calculate name similarity
    cleaned_df = (apply_category_mapping(cleaned_df)
                 .transform(calculate_similarity))
    
    return cleaned_df

# Process the data
cleaned_product_category = clean_product_categories(product_category)

# Display results and analysis
print("Singular/Plural Inconsistencies:")
cleaned_product_category.filter(~col("is_plural_consistent")).select(
    "product_category_name_clean", 
    "product_category_name_english_clean",
    "product_category_name_english_title",
    "is_original_plural", 
    "is_english_plural"
).show(truncate=False)

# Calculate and display metrics
total_rows = cleaned_product_category.count()
cleaned_product_category.agg(
    sum_(col("is_plural_consistent").cast("int")).alias("consistent_count"),
    sum_(when(col("name_similarity") == 0, 1).otherwise(0)).alias("exact_match_count"),
    (sum_(col("is_plural_consistent").cast("int")) / count_("*") * 100).alias("plural_consistency_percentage"),
    (sum_(when(col("name_similarity") == 0, 1).otherwise(0)) / count_("*") * 100).alias("exact_match_percentage")
).show()

print("Categories with high name dissimilarity:")
cleaned_product_category.filter(col("name_similarity") > 15).orderBy(
    col("name_similarity").desc()
).select(
    "product_category_name_clean",
    "product_category_name_english_clean",
    "product_category_name_english_title",
    "name_similarity"
).show(10, truncate=False)

# Display final result
display(cleaned_product_category.limit(10))

print("\nCleaning process completed.")

Singular/Plural Inconsistencies:
+---------------------------+-----------------------------------+-----------------------------------+------------------+-----------------+
|product_category_name_clean|product_category_name_english_clean|product_category_name_english_title|is_original_plural|is_english_plural|
+---------------------------+-----------------------------------+-----------------------------------+------------------+-----------------+
+---------------------------+-----------------------------------+-----------------------------------+------------------+-----------------+

+----------------+-----------------+-----------------------------+----------------------+
|consistent_count|exact_match_count|plural_consistency_percentage|exact_match_percentage|
+----------------+-----------------+-----------------------------+----------------------+
|              71|               16|                        100.0|    22.535211267605636|
+----------------+-----------------+--------------

product_category_name,product_category_name_english,product_category_name_clean,product_category_name_english_clean,product_category_name_english_title,is_original_plural,is_english_plural,is_plural_consistent,normalized_pt,normalized_en,name_similarity
beleza_saude,health_beauty,beleza_saude,health_beauty,Health Beauty,false,false,true,beleza_saude,health_beauty,9
informatica_acessorios,computers_accessories,informatica_acessorios,computers_accessories,Computers Accessories,true,true,true,informatica_acessorios,computers_accessories,11
automotivo,auto,automotivo,auto,Auto,false,false,true,automotivo,auto,6
cama_mesa_banho,bed_bath_table,cama_mesa_banho,bed_bath_table,Bed Bath Table,false,false,true,cama_mesa_banho,bed_bath_table,12
moveis_decoracao,furniture_decor,moveis_decoracao,furniture_decor,Furniture Decor,false,false,true,furniture_decoracao,furniture_decor,4
esporte_lazer,sports_leisure,esporte_lazer,sports_leisure,Sports Leisure,false,false,true,esporte_lazer,sports_leisure,7
perfumaria,perfumery,perfumaria,perfumery,Perfumery,false,false,true,perfumaria,perfumery,3
utilidades_domesticas,housewares,utilidades_domesticas,housewares,Housewares,true,true,true,housewares,housewares,0
telefonia,telephony,telefonia,telephony,Telephony,false,false,true,telefonia,telephony,4
relogios_presentes,watches_gifts,relogios_presentes,watches_gifts,Watches Gifts,true,true,true,relogios_presentes,watches_gifts,14



Cleaning process completed.


### Review the cleaned_product_category dataset

In [0]:
# Select only the required columns and create final DataFrame
cleaned_product_categories = cleaned_product_category.select(
    "product_category_name_clean",
    "product_category_name_english_title"
)

# Verify final schema
print("\nFinal Schema:")
cleaned_product_categories.printSchema()

# Show sample of final dataset
print("\nSample of final cleaned dataset:")
cleaned_product_categories.show(5, truncate=False)

# Print final record count
print(f"\nTotal records in cleaned dataset: {cleaned_product_categories.count():,}")


Final Schema:
root
 |-- product_category_name_clean: string (nullable = true)
 |-- product_category_name_english_title: string (nullable = true)


Sample of final cleaned dataset:
+---------------------------+-----------------------------------+
|product_category_name_clean|product_category_name_english_title|
+---------------------------+-----------------------------------+
|beleza_saude               |Health Beauty                      |
|informatica_acessorios     |Computers Accessories              |
|automotivo                 |Auto                               |
|cama_mesa_banho            |Bed Bath Table                     |
|moveis_decoracao           |Furniture Decor                    |
+---------------------------+-----------------------------------+
only showing top 5 rows


Total records in cleaned dataset: 71


### Save the cleaned_product_category to a parquet file

In [0]:
# Define the output path for final cleaned and translated dataset
output_path = "/mnt/olist-store-data/transformed-data/olist_product_category_cleaned_dataset_final_v2.0.parquet"
temp_path = "/mnt/olist-store-data/transformed-data/temp_parquet_output"

try:
    # Remove existing directories if they exist
    dbutils.fs.rm(output_path, recurse=True)
    dbutils.fs.rm(temp_path, recurse=True)

    # Save as a single Parquet file using temporary directory
    (cleaned_product_categories
     .repartition(1)  # Force to a single partition
     .write
     .mode("overwrite")
     .parquet(temp_path))

    # Find the Parquet file in temp directory
    temp_files = dbutils.fs.ls(temp_path)
    parquet_file = [f.path for f in temp_files if f.path.endswith(".parquet")][0]
    
    # Move to final location
    dbutils.fs.mv(parquet_file, output_path)
    
    # Clean up temp directory
    dbutils.fs.rm(temp_path, recurse=True)

    # Verify the saved Parquet file
    verified_df = spark.read.parquet(output_path)
    print("\nVerification of saved Parquet file:")
    print(f"Number of rows in saved Parquet file: {verified_df.count():,}")
    print("\nSample of saved data:")
    display(verified_df.limit(5))

    # Verify it's a single file
    if len(dbutils.fs.ls(output_path)) == 1:
        print("\nSuccessfully saved as a single Parquet file.")
    else:
        print("\nWarning: Multiple files were created.")

except Exception as e:
    print(f"Error saving dataset: {str(e)}")
    # Clean up temp directory in case of failure
    dbutils.fs.rm(temp_path, recurse=True)
    raise
finally:
    # Unpersist cached DataFrame
    cleaned_product_categories.unpersist()


Verification of saved Parquet file:
Number of rows in saved Parquet file: 71

Sample of saved data:


product_category_name_clean,product_category_name_english_title
beleza_saude,Health Beauty
informatica_acessorios,Computers Accessories
automotivo,Auto
cama_mesa_banho,Bed Bath Table
moveis_decoracao,Furniture Decor



Successfully saved as a single Parquet file.


### Read products dataset from raw-data folder

In [0]:
products = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/olist-store-data/raw-data/olist_products_dataset.csv")

In [0]:
products.show(10)
products.printSchema()
display(products.limit(10))

+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|          product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|1e9e8ef04dbcff454...|           perfumaria|                 40|                       287|                 1|             225|               16|               10|              14|
|3aa071139cb16b67c...|                artes|                 44|                       276|                 1|            1000|               30|               18|              20|
|96bd76ec8810374ed...|        esporte_lazer|                 46|                       250|    

product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13
41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,60,745,1,200,38,5,11
732bd381ad09e530fe0a5f457d81becb,cool_stuff,56,1272,4,18350,70,24,44
2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,56,184,2,900,40,8,40
37cc742be07708b53a98702e77a21a02,eletrodomesticos,57,163,1,400,27,13,17
8c92109888e8cdf9d66dc7e463025574,brinquedos,36,1156,1,600,17,10,12


### Data cleaning for products dataset
- **Step 1: Data Loading and Initial Processing**<br>
The function clean_products_dataset starts by loading two datasets:
  - The main products dataset from a CSV file
  - A cleaned product categories dataset from a Parquet file
These datasets are joined to replace the original category names with English titles. The function also renames misspelled columns and performs initial data quality checks.

- **Step 2: Data Quality Assessment**:<br>
The code performs several data quality checks:
  - Missing Values Analysis: It calculates and displays the count and percentage of missing values for each column.
  - Duplicate Check: It identifies and reports the number of duplicate product IDs.

- **Step 3: Data Cleaning and Transformation**:<br>
The cleaning process involves several steps:
  - Basic Cleaning: Removes records with null values in critical fields.
  - Text Standardization: Converts category names to lowercase and trims whitespace.
  - Numeric Value Handling: Replaces zero or negative values with null in numeric columns.
  - Derived Metrics: Calculates product volume and density.

- **Step 4: Feature Engineering**:<br>
The code creates several new features to enhance the dataset:
  - **Volume Tier**:<br>Categorizes products based on their volume (Extra Small to Extra Large). This categorization helps in understanding the size distribution of products and can be useful for inventory management and shipping considerations. It's calculated using quartiles of the product_volume_cm3 field:
    - Extra Small: <= q1_volume/2
    - Small: <= q1_volume
    - Medium Small: <= median_volume
    - Medium Large: <= q3_volume
    - Large: <= q3_volume*2
    - Extra Large: > q3_volume*2

  - **Weight Category**:<br>Classifies products by weight (Ultra Light to Extra Heavy). This categorization is valuable for shipping calculations, warehouse organization, and logistics planning. The weight_category classifies products based on their weight:
    - Ultra Light: <= 250g
    - Light: <= q1_weight
    - Medium Light: <= median_weight
    - Medium Heavy: <= q3_weight
    - Heavy: <= q3_weight*2
    - Extra Heavy: > q3_weight*2  

  - **Product Complexity**:<br>Assigns complexity levels based on description length, photo quantity, volume, and weight. This metric can be useful for customer support, product management, and marketing strategies. The product_complexity field assesses the overall complexity of a product based on multiple factors:
    - Very High: Long description, many photos, large volume, heavy weight
    - High: Above average description length, multiple photos, above average volume
    - Medium: Default category
    - Medium Low: Below average description length, few photos, below average volume
    - Low: Short description, single photo, small volume, light weight

  - **Shipping Complexity**:<br>Categorizes products as Simple, Standard, or Complex based on weight and dimensions. This categorization is crucial for logistics planning, shipping cost estimation, and warehouse management. The shipping_complexity categorizes products based on their shipping requirements:
    - Complex: Heavy weight, large volume, or any dimension > 100cm
    - Simple: Light weight and small volume
    - Standard: Everything else

  - **Category Group**:<br>Groups products into broader categories like "Home & Decor", "Health & Fashion", etc. This grouping allows for higher-level analysis of product trends and can be useful for marketing, inventory management, and strategic decision-making. The category_group field aggregates similar product categories into broader groups:
    - Home & Decor
    - Health & Fashion
    - Electronics & Tech
    - Kids & Baby
    - Gifts & Personal
    - Food & Beverage
    - Tools & Construction
    - Other

- **Step 5: Statistical Analysis and Reporting**:<br>
The code calculates and reports various statistics:
  - Quartiles for volume, weight, and description length.
  - Distribution of products across the newly created categories.

- **Step 6: Final Cleaning and Output**:
The function concludes by:
  - Reporting the final record count and data retention rate.
  - Displaying the cleaned dataset's schema.
  - Showing a sample of the cleaned data.

In [0]:
# This script performs data cleaning and analysis on the products dataset.
# It standardizes product categories, handles missing values, and adds derived features.

# Import libraries
from pyspark.sql.functions import (
    col, sum, count, when, isnan, 
    regexp_replace, lower, trim, upper, concat,
    avg, stddev, min, max, round,
    array, explode, concat_ws, percentile_approx, lit, udf
)
from pyspark.sql.types import IntegerType, StringType

def sentence_case(text):
    if text:
        words = text.split()
        return ' '.join(word.capitalize() for word in words)
    return None

sentence_case_udf = udf(sentence_case, StringType())

def clean_products_dataset(spark):
    """
    This function performs comprehensive data cleaning and analysis on the sellers dataset.
    It standardizes city and state names, adds derived features, and generates various analytical reports.
    """
    try:
        # Load the sellers dataset from CSV
        print("Loading products dataset...")
        products = spark.read.format("csv").option("header","true").option("inferSchema","true")\
            .load("/mnt/olist-store-data/raw-data/olist_products_dataset.csv")
        
        cleaned_product_categories = spark.read.parquet("/mnt/olist-store-data/transformed-data/olist_product_category_cleaned_dataset_final_v2.0.parquet")

        products = products.join(
            cleaned_product_categories.select("product_category_name_clean", "product_category_name_english_title"),
            products["product_category_name"] == cleaned_product_categories["product_category_name_clean"],
            "left"
        )
        products = products.drop("product_category_name", "product_category_name_clean") \
                   .withColumnRenamed("product_category_name_english_title", "product_category_name")
        
        products = products.withColumnRenamed("product_name_lenght", "product_name_length") \
                           .withColumnRenamed("product_description_lenght", "product_description_length")
        
        initial_count = products.count()
        
        print("\nInitial dataset information:")
        print(f"Number of records: {initial_count:,}")
        print(f"Number of columns: {len(products.columns)}")
        
        print("\nMissing values analysis:")
        missing_values = products.select([
            sum(col(c).isNull().cast("int")).alias(c) for c in products.columns
        ])
        
        for column in products.columns:
            missing_count = missing_values.collect()[0][column]
            missing_percentage = (missing_count / initial_count) * 100
            print(f"{column}: {missing_count:,} missing values ({missing_percentage:.2f}%)")
        
        print("\nMissing values count:")
        missing_values.show()
        
        print("\nChecking for duplicate products...")
        duplicates = products.groupBy("product_id").count().filter(col("count") > 1)
        duplicate_count = duplicates.count()
        duplicate_percentage = (duplicate_count / initial_count) * 100
        print(f"Number of duplicate product_ids: {duplicate_count:,} ({duplicate_percentage:.2f}%)")
        
        print("\nStarting data cleaning process...")
        
        cleaned_products = products.filter(
            col("product_id").isNotNull() &
            col("product_category_name").isNotNull()
        )
        
        # Convert product_category_name to sentence case
        cleaned_products = cleaned_products.withColumn("product_category_name", sentence_case_udf(trim(col("product_category_name")))
        )

        
        numeric_columns = [
            "product_name_length",
            "product_description_length",
            "product_photos_qty",
            "product_weight_g",
            "product_length_cm",
            "product_height_cm",
            "product_width_cm"
        ]
        
        for column in numeric_columns:
            cleaned_products = cleaned_products.withColumn(
                column,
                when(col(column) <= 0, None).otherwise(col(column))
            )
        
        cleaned_products = cleaned_products.withColumn(
            "product_volume_cm3",
            col("product_length_cm") * col("product_height_cm") * col("product_width_cm")
        ).withColumn(
            "product_density_g_cm3",
            round(col("product_weight_g") / col("product_volume_cm3"), 3)
        )
        
        print("\nCalculating data statistics...")
        stats = cleaned_products.select(
            percentile_approx("product_volume_cm3", array(lit(0.25), lit(0.5), lit(0.75))).alias("volume_percentiles"),
            percentile_approx("product_weight_g", array(lit(0.25), lit(0.5), lit(0.75))).alias("weight_percentiles"),
            percentile_approx("product_description_length", array(lit(0.25), lit(0.5), lit(0.75))).alias("description_percentiles")
        ).collect()[0]
        
        q1_volume, median_volume, q3_volume = stats["volume_percentiles"]
        q1_weight, median_weight, q3_weight = stats["weight_percentiles"]
        q1_desc, median_desc, q3_desc = stats["description_percentiles"]
        
        cleaned_products = cleaned_products.withColumn(
            "volume_tier",
            when(col("product_volume_cm3") <= q1_volume/2, "Extra Small")
            .when(col("product_volume_cm3") <= q1_volume, "Small")
            .when(col("product_volume_cm3") <= median_volume, "Medium Small")
            .when(col("product_volume_cm3") <= q3_volume, "Medium Large")
            .when(col("product_volume_cm3") <= q3_volume*2, "Large")
            .otherwise("Extra Large")
        )
        
        cleaned_products = cleaned_products.withColumn(
            "weight_category",
            when(col("product_weight_g") <= 250, "Ultra Light")
            .when(col("product_weight_g") <= q1_weight, "Light")
            .when(col("product_weight_g") <= median_weight, "Medium Light")
            .when(col("product_weight_g") <= q3_weight, "Medium Heavy")
            .when(col("product_weight_g") <= q3_weight*2, "Heavy")
            .otherwise("Extra Heavy")
        )
        
        cleaned_products = cleaned_products.withColumn(
            "product_complexity",
            when(
                (col("product_description_length") > q3_desc) & 
                (col("product_photos_qty") > 2) & 
                (col("product_volume_cm3") > q3_volume) &
                (col("product_weight_g") > q3_weight),
                "Very High"
            )
            .when(
                (col("product_description_length") > median_desc) & 
                (col("product_photos_qty") > 1) & 
                (col("product_volume_cm3") > median_volume),
                "High"
            )
            .when(
                (col("product_description_length") < q1_desc) & 
                (col("product_photos_qty") == 1) & 
                (col("product_volume_cm3") < q1_volume) &
                (col("product_weight_g") < q1_weight),
                "Low"
            )
            .when(
                (col("product_description_length") < median_desc) & 
                (col("product_photos_qty") <= 2) & 
                (col("product_volume_cm3") < median_volume),
                "Medium Low"
            )
            .otherwise("Medium")
        )
        
        cleaned_products = cleaned_products.withColumn(
            "shipping_complexity",
            when(
                (col("product_weight_g") > q3_weight) | 
                (col("product_volume_cm3") > q3_volume) |
                (col("product_length_cm") > 100) |
                (col("product_height_cm") > 100) |
                (col("product_width_cm") > 100),
                "Complex"
            )
            .when(
                (col("product_weight_g") <= q1_weight) & 
                (col("product_volume_cm3") <= q1_volume),
                "Simple"
            )
            .otherwise("Standard")
        )
        
        cleaned_products = cleaned_products.withColumn(
        "category_group",
            when(
                lower(col("product_category_name")).isin(
                    "bed bath table", "furniture decor", "home construction", 
                    "housewares", "office furniture", "furniture living room",
                    "kitchen dining laundry garden furniture", "home comfort"
                ), 
                "Home & Decor"
            )
            .when(
                lower(col("product_category_name")).isin(
                    "sports leisure", "health beauty", "fashion bags accessories", 
                    "fashion shoes", "fashion male clothing", "luggage accessories"
                ), 
                "Health & Fashion"
            )
            .when(
                lower(col("product_category_name")).isin(
                    "computers accessories", "telephony", "electronics", "pc gamer",
                    "tablets printing image", "fixed telephony", "home appliances",
                    "home appliances 2", "small appliances", "air conditioning"
                ), 
                "Electronics & Tech"
            )
            .when(
                lower(col("product_category_name")).isin(
                    "toys", "baby", "fashion children clothes"
                ), 
                "Kids & Baby"
            )
            .when(
                lower(col("product_category_name")).isin(
                    "watches gifts", "perfumery", "art", "christmas articles"
                ), 
                "Gifts & Personal"
            )
            .when(
                lower(col("product_category_name")).isin(
                    "food", "drinks", "food drink"
                ),
                "Food & Beverage"
            )
            .when(
                lower(col("product_category_name")).isin(
                    "garden tools", "construction tools construction",
                    "construction tools garden", "construction tools safety",
                    "construction tools lights"
                ),
                "Tools & Construction"
            )
            .otherwise("Other")
        )
        
        print("\nVolume Tier Distribution:")
        cleaned_products.groupBy("volume_tier").count()\
            .withColumn("percentage", round(col("count") / cleaned_products.count() * 100, 2))\
            .orderBy("count", ascending=False).show()
        
        print("\nWeight Category Distribution:")
        cleaned_products.groupBy("weight_category").count()\
            .withColumn("percentage", round(col("count") / cleaned_products.count() * 100, 2))\
            .orderBy("count", ascending=False).show()
        
        print("\nProduct Complexity Distribution:")
        cleaned_products.groupBy("product_complexity").count()\
            .withColumn("percentage", round(col("count") / cleaned_products.count() * 100, 2))\
            .orderBy("count", ascending=False).show()
        
        print("\nShipping Complexity Distribution:")
        cleaned_products.groupBy("shipping_complexity").count()\
            .withColumn("percentage", round(col("count") / cleaned_products.count() * 100, 2))\
            .orderBy("count", ascending=False).show()
        
        print("\nCategory Group Distribution:")
        cleaned_products.groupBy("category_group").count()\
            .withColumn("percentage", round(col("count") / cleaned_products.count() * 100, 2))\
            .orderBy("count", ascending=False).show()
        
        final_count = cleaned_products.count()
        removed_count = initial_count - final_count
        retention_rate = (final_count / initial_count) * 100
        removal_rate = (removed_count / initial_count) * 100
        
        print("\nCleaning Summary:")
        print(f"Original record count: {initial_count:,}")
        print(f"Cleaned record count: {final_count:,}")
        print(f"Records removed: {removed_count:,}")
        print(f"Data retention rate: {retention_rate:.2f}%")
        print(f"Data removal rate: {removal_rate:.2f}%")
        
        print("\nCleaned Dataset Schema:")
        cleaned_products.printSchema()
        
        print("\nSample of Cleaned Data:")
        cleaned_products.select(
            "product_id", "product_category_name", "category_group", "product_complexity", 
            "shipping_complexity", "volume_tier", "weight_category"
        ).show(5, truncate=True)
        
        display(cleaned_products.limit(10))
        print("\nCleaning process completed.")
        return cleaned_products
        
    except Exception as e:
        print(f"\nError in data cleaning process: {str(e)}")
        raise

if __name__ == "__main__":
    try:
        cleaned_products = clean_products_dataset(spark)
    except Exception as e:
        print(f"Failed to clean products dataset: {str(e)}")



Loading products dataset...

Initial dataset information:
Number of records: 32,951
Number of columns: 9

Missing values analysis:
product_id: 0 missing values (0.00%)
product_name_length: 610 missing values (1.85%)
product_description_length: 610 missing values (1.85%)
product_photos_qty: 610 missing values (1.85%)
product_weight_g: 2 missing values (0.01%)
product_length_cm: 2 missing values (0.01%)
product_height_cm: 2 missing values (0.01%)
product_width_cm: 2 missing values (0.01%)
product_category_name: 623 missing values (1.89%)

Missing values count:
+----------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------+
|product_id|product_name_length|product_description_length|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name|
+----------+-------------------+--------------------------+------------------+---------------

product_id,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_volume_cm3,product_density_g_cm3,volume_tier,weight_category,product_complexity,shipping_complexity,category_group
1e9e8ef04dbcff4541ed26657ea517e5,40,287,1,225,16,10,14,Perfumery,2240,0.1,Small,Ultra Light,Low,Simple,Gifts & Personal
3aa071139cb16b67ca9e5dea641aaa2f,44,276,1,1000,30,18,20,Art,10800,0.093,Medium Large,Medium Heavy,Medium,Standard,Gifts & Personal
96bd76ec8810374ed1b65e291975717f,46,250,1,154,18,9,15,Sports Leisure,2430,0.063,Small,Ultra Light,Low,Simple,Health & Fashion
cef67bcfe19066a932b7673e239eb23d,27,261,1,371,26,4,26,Baby,2704,0.137,Small,Medium Light,Medium Low,Standard,Kids & Baby
9dc1a7de274444849c219cff195d0b71,37,402,4,625,20,17,13,Housewares,4420,0.141,Medium Small,Medium Light,Medium,Standard,Home & Decor
41d3672d4792049fa1779bb35283ed13,60,745,1,200,38,5,11,Musical Instruments,2090,0.096,Small,Ultra Light,Medium,Simple,Other
732bd381ad09e530fe0a5f457d81becb,56,1272,4,18350,70,24,44,Cool Stuff,73920,0.248,Extra Large,Extra Heavy,Very High,Complex,Other
2548af3e6e77a690cf3eb6368e9ab61e,56,184,2,900,40,8,40,Furniture Decor,12800,0.07,Medium Large,Medium Heavy,Medium,Standard,Home & Decor
37cc742be07708b53a98702e77a21a02,57,163,1,400,27,13,17,Home Appliances,5967,0.067,Medium Small,Medium Light,Medium Low,Standard,Electronics & Tech
8c92109888e8cdf9d66dc7e463025574,36,1156,1,600,17,10,12,Toys,2040,0.294,Small,Medium Light,Medium,Standard,Kids & Baby



Cleaning process completed.


### Review the cleaned_products_withCalculations dataset

In [0]:
# Verify final schema
print("\nFinal Schema:")
cleaned_products.printSchema()

# Show sample of final dataset
print("\nSample of final cleaned dataset:")
cleaned_products.show(5, truncate=False)

# Print final record count
print(f"\nTotal records in cleaned dataset: {cleaned_products.count():,}")


Final Schema:
root
 |-- product_id: string (nullable = true)
 |-- product_name_length: integer (nullable = true)
 |-- product_description_length: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_volume_cm3: integer (nullable = true)
 |-- product_density_g_cm3: double (nullable = true)
 |-- volume_tier: string (nullable = false)
 |-- weight_category: string (nullable = false)
 |-- product_complexity: string (nullable = false)
 |-- shipping_complexity: string (nullable = false)
 |-- category_group: string (nullable = false)


Sample of final cleaned dataset:
+--------------------------------+-------------------+--------------------------+------------------+----------------+-----------------+---------

### Save the clean_products_withCalculations dataset in parquet

In [0]:
# Define the output path for final cleaned and translated dataset
output_path = "/mnt/olist-store-data/transformed-data/olist_products_cleaned_dataset_withCalculations.parquet"
temp_path = "/mnt/olist-store-data/transformed-data/temp_parquet_output"

try:
    # Remove existing directories if they exist
    dbutils.fs.rm(output_path, recurse=True)
    dbutils.fs.rm(temp_path, recurse=True)

    # Save as a single Parquet file using temporary directory
    (cleaned_products
     .repartition(1)  # Force to a single partition
     .write
     .mode("overwrite")
     .parquet(temp_path))

    # Find the Parquet file in temp directory
    temp_files = dbutils.fs.ls(temp_path)
    parquet_file = [f.path for f in temp_files if f.path.endswith(".parquet")][0]
    
    # Move to final location
    dbutils.fs.mv(parquet_file, output_path)
    
    # Clean up temp directory
    dbutils.fs.rm(temp_path, recurse=True)

    # Verify the saved Parquet file
    verified_df = spark.read.parquet(output_path)
    print("\nVerification of saved Parquet file:")
    print(f"Number of rows in saved Parquet file: {verified_df.count():,}")
    print("\nSample of saved data:")
    display(verified_df.limit(5))

    # Verify it's a single file
    if len(dbutils.fs.ls(output_path)) == 1:
        print("\nSuccessfully saved as a single Parquet file.")
    else:
        print("\nWarning: Multiple files were created.")

except Exception as e:
    print(f"Error saving dataset: {str(e)}")
    # Clean up temp directory in case of failure
    dbutils.fs.rm(temp_path, recurse=True)
    raise
finally:
    # Unpersist cached DataFrame
    cleaned_products.unpersist()


Verification of saved Parquet file:
Number of rows in saved Parquet file: 32,328

Sample of saved data:


product_id,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_volume_cm3,product_density_g_cm3,volume_tier,weight_category,product_complexity,shipping_complexity,category_group
1e9e8ef04dbcff4541ed26657ea517e5,40,287,1,225,16,10,14,Perfumery,2240,0.1,Small,Ultra Light,Low,Simple,Gifts & Personal
3aa071139cb16b67ca9e5dea641aaa2f,44,276,1,1000,30,18,20,Art,10800,0.093,Medium Large,Medium Heavy,Medium,Standard,Gifts & Personal
96bd76ec8810374ed1b65e291975717f,46,250,1,154,18,9,15,Sports Leisure,2430,0.063,Small,Ultra Light,Low,Simple,Health & Fashion
cef67bcfe19066a932b7673e239eb23d,27,261,1,371,26,4,26,Baby,2704,0.137,Small,Medium Light,Medium Low,Standard,Kids & Baby
9dc1a7de274444849c219cff195d0b71,37,402,4,625,20,17,13,Housewares,4420,0.141,Medium Small,Medium Light,Medium,Standard,Home & Decor



Successfully saved as a single Parquet file.


### Review the cleaned_products dataset with selected columns

In [0]:
from pyspark.sql.functions import col

def preview_products(spark):
    # Load the existing cleaned products dataset
    cleaned_products = spark.read.parquet("/mnt/olist-store-data/transformed-data/olist_products_cleaned_dataset_withCalculations.parquet")

    # Create DataFrame with selected columns
    cleaned_selected_products = cleaned_products.select(
        "product_id",
        "product_category_name",
        "product_name_length",
        "product_description_length",
        "product_photos_qty",
        "product_weight_g",
        "product_length_cm",
        "product_height_cm",
        "product_width_cm"
    )

    # Verify final schema
    print("\nFinal Schema:")
    cleaned_selected_products.printSchema()

    # Show sample of final dataset
    print("\nSample of final cleaned dataset:")
    display(cleaned_selected_products.limit(5))

    # Print final record count
    print(f"\nTotal records in cleaned dataset: {cleaned_selected_products.count():,}")

# Call the function with the spark session
preview_products(spark)


Final Schema:
root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_length: integer (nullable = true)
 |-- product_description_length: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)


Sample of final cleaned dataset:


product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1e9e8ef04dbcff4541ed26657ea517e5,Perfumery,40,287,1,225,16,10,14
3aa071139cb16b67ca9e5dea641aaa2f,Art,44,276,1,1000,30,18,20
96bd76ec8810374ed1b65e291975717f,Sports Leisure,46,250,1,154,18,9,15
cef67bcfe19066a932b7673e239eb23d,Baby,27,261,1,371,26,4,26
9dc1a7de274444849c219cff195d0b71,Housewares,37,402,4,625,20,17,13



Total records in cleaned dataset: 32,328


### Save the cleaned_products with selected columns in parquet file

In [0]:
# Define the output path for final cleaned and translated dataset
output_path = "/mnt/olist-store-data/transformed-data/olist_products_cleaned_dataset_v2.0.parquet"
temp_path = "/mnt/olist-store-data/transformed-data/temp_products_parquet"

try:
    # Remove existing directories if they exist
    dbutils.fs.rm(output_path, recurse=True)
    dbutils.fs.rm(temp_path, recurse=True)

    # Save as a single Parquet file using temporary directory
    (selected_products
     .repartition(1)  # Force to a single partition
     .write
     .mode("overwrite")
     .parquet(temp_path))

    # Find the Parquet file in temp directory
    temp_files = dbutils.fs.ls(temp_path)
    parquet_file = [f.path for f in temp_files if f.path.endswith(".parquet")][0]
    
    # Move to final location
    dbutils.fs.mv(parquet_file, output_path)
    
    # Clean up temp directory
    dbutils.fs.rm(temp_path, recurse=True)

    # Verify the saved Parquet file
    verified_df = spark.read.parquet(output_path)
    print("\nVerification of saved Parquet file:")
    print(f"Number of rows in saved Parquet file: {verified_df.count():,}")
    print("\nSample of saved data:")
    display(verified_df.limit(5))

    # Verify it's a single file
    if len(dbutils.fs.ls(output_path)) == 1:
        print("\nSuccessfully saved as a single Parquet file.")
    else:
        print("\nWarning: Multiple files were created.")

except Exception as e:
    print(f"Error saving dataset: {str(e)}")
    # Clean up temp directory in case of failure
    dbutils.fs.rm(temp_path, recurse=True)
    raise
finally:
    # Unpersist cached DataFrame
    selected_products.unpersist()


Verification of saved Parquet file:
Number of rows in saved Parquet file: 32,328

Sample of saved data:


product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
1e9e8ef04dbcff4541ed26657ea517e5,Perfumery,40,287,1,225,16,10,14
3aa071139cb16b67ca9e5dea641aaa2f,Art,44,276,1,1000,30,18,20
96bd76ec8810374ed1b65e291975717f,Sports Leisure,46,250,1,154,18,9,15
cef67bcfe19066a932b7673e239eb23d,Baby,27,261,1,371,26,4,26
9dc1a7de274444849c219cff195d0b71,Housewares,37,402,4,625,20,17,13



Successfully saved as a single Parquet file.
